In [2]:
import torch
from torch.autograd import Variable

import pyro
import pyro.distributions as dist

In [5]:
# draw a sample x from unit normal distribution

mu = Variable(torch.zeros(1)) # mean zero
sigma = Variable(torch.ones(1)) # unit variance
x = dist.normal(mu, sigma) # x is sample from N(0,1)
print(x)

Variable containing:
-0.5316
[torch.FloatTensor of size 1]



In [6]:
# score sample x, i.e. compute log prob according to N(0,1)

log_p_x = dist.normal.log_pdf(x, mu, sigma)
print(log_p_x)

Variable containing:
-1.0602
[torch.FloatTensor of size 1]



In [8]:
# named sample

x = pyro.sample("my_sample", dist.normal, mu, sigma )
print(x)

Variable containing:
-0.2348
[torch.FloatTensor of size 1]



In [12]:
# 맑은/흐린 날씨에서의 온도

def weather():
    cloudy = pyro.sample("cloudy", dist.bernoulli, Variable(torch.Tensor([0.3])))
    cloudy = 'cloudy' if cloudy.data[0] == 1.0 else 'sunny'
    mean_temp = {'cloudy': [55.0], 'sunny': [75.0]}[cloudy]
    sigma_temp = {'cloudy': [10.0], 'sunny': [15.0]}[cloudy]
    temp = pyro.sample('temp', dist.normal, 
                       Variable(torch.Tensor(mean_temp)),
                       Variable(torch.Tensor(sigma_temp))
                      )
    return cloudy, temp.data[0]
for _ in range(3):
    print(weather())

('sunny', 81.83702850341797)
('sunny', 86.78369903564453)
('cloudy', 51.43877029418945)


In [18]:
# 날씨/온도에 따른 아이스크림 장사

def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = [200] if cloudy == 'sunny' and temp > 80.0 else [50]
    ice_cream = pyro.sample('ice_cream', dist.normal,
                            Variable(torch.Tensor(expected_sales)),
                            Variable(torch.Tensor([10.0])))
    return cloudy, temp, ice_cream.data[0]

for _ in range(3):
    print(ice_cream_sales())

('cloudy', 55.03554153442383, 44.32622146606445)
('sunny', 92.1712875366211, 195.0894775390625)
('sunny', 66.23624420166016, 70.21913146972656)


In [27]:
# recursive function that terminate nondeterministically

def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), dist.bernoulli, p)
    if torch.equal(x.data, torch.zeros(1)):
        return x
    else:
        return x + geometric(p, t+1)

print(geometric(Variable(torch.Tensor([0.7]))))

Variable containing:
 6
[torch.FloatTensor of size 1]



In [28]:
# stochastic function that accept stochastic function as input, producing stochastic function as output

def normal_product(mu, sigma):
    z1 = pyro.sample("z1", dist.normal, mu, sigma)
    z2 = pyro.sample("z2", dist.normal, mu, sigma)
    y = z1 * z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", dist.normal,
                            Variable(torch.zeros(1)),
                            Variable(torch.ones(1)))
    fn = lambda sigma: normal_product(mu_latent, sigma)
    return fn

print(make_normal_normal()(Variable(torch.ones(1))))

Variable containing:
 0.6565
[torch.FloatTensor of size 1]

